In [1]:
from model import *

In [ ]:
#Parameter initialization
model = Trans_unet()
        
model_checkpoint = ModelCheckpoint('3D_TransU-Net.hdf5', monitor='val_loss',verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, min_lr=0)
callbacks_list = [model_checkpoint, reduce_lr]

model.summary()

In [ ]:
#Transfer learning
for layer in model.layers[ :98]: 
    layer.trainable = False

for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

model.compile(optimizer = Adam(), loss = MeanAbsoluteError())

In [3]:
#Extract data (n, 128, 128, 8, 4)
x_data_aug = np.load('x_data_aug.npy')
y_data_nor_aug = np.load('y_data_nor_aug.npy')

x_data_val = np.load('x_data_val.npy')
y_data_val_nor = np.load('y_data_val_nor.npy')

In [ ]:
history = model.fit(x_data_aug,
                    y_data_nor_aug,
                    validation_data = (x_data_val, y_data_val_nor),
                    batch_size = 4,
                    epochs = 50,
                    verbose = 1,
                    callbacks = callbacks_list,
                    shuffle = True,
                    steps_per_epoch = 250)